In [1]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
import numpy as np

In [2]:
def print_rota_otima(Modelo):
    lista_produtores_ordenados = [(i, value(Modelo.y[i])) for i, v in Modelo.y.items() 
                                  if value(Modelo.y[i]) != 0 or i == 0]
    lista_produtores_ordenados.sort(key=lambda x: x[1])

    print('Rota ótima:')
    print(' -> '.join(str(p) for p, _ in lista_produtores_ordenados))


In [3]:

def calcula_tempo_coleta(i):
    # Tempo necessário para realizar a coleta (em minutos)
    tempo_coleta = i['demanda'] / velocidade_coleta
    tempo_coleta = tempo_coleta / 60

    return tempo_coleta

In [4]:
def calcula_distancia_produtores(p1, p2):
    return np.sqrt((p1['posição']['leste'] - p2['posição']['leste'])**2 + 
                   (p1['posição']['norte'] - p2['posição']['norte'])**2)*10

# Planejamento de Coleta de Leite

---
### Descrição do Problema

Vamos considerar uma variante do Problema do Caixeiro Viajante (PCV) para um grafo específico associado ao problema da coleta de leite, que contém 21 vértices conforme as localizações dadas na proposta do exercício.

### Formulação do Problema

Nesta variante do PCV, cada cliente (vértice) só pode ser visitado dentro de janelas de tempo específicas do dia. Assumimos que a capacidade do veículo é suficiente para coletar todo o leite demandado, de forma que todos os clientes devem ser visitados em um único circuito Hamiltoniano. No entanto, a coleta do leite no cliente (i) só pode começar dentro da janela de tempo específica [ei, li]. Se o veículo chegar antes de (ei), deve esperar até que a janela de tempo comece. A coleta pode terminar após \(li), mas não pode começar antes de (ei).

Para esta instância, consideramos as seguintes suposições:
1. A coleta de cada 1000 litros de leite leva 1 minuto.
2. A velocidade média de deslocamento do veículo é de 80 km/h.

[Mais detalhes do problema](exercicio-coleta-leite-janela-de-tempo.pdf)


## Parâmetros para a modelagem

In [5]:
produtores = [
    {"id": 1, "posição": {"leste": 0, "norte": 0}, "frequência": "d", "demanda": 0, "janela_de_tempo": {"início": 0, "fim": 24}},
    {"id": 2, "posição": {"leste": -3, "norte": 3}, "frequência": "d", "demanda": 5, "janela_de_tempo": {"início": 13, "fim": 18}},
    {"id": 3, "posição": {"leste": 1, "norte": 11}, "frequência": "d", "demanda": 4, "janela_de_tempo": {"início": 2, "fim": 8}},
    {"id": 4, "posição": {"leste": 4, "norte": 7}, "frequência": "d", "demanda": 3, "janela_de_tempo": {"início": 11, "fim": 17}},
    {"id": 5, "posição": {"leste": -5, "norte": 9}, "frequência": "d", "demanda": 6, "janela_de_tempo": {"início": 8, "fim": 11}},
    {"id": 6, "posição": {"leste": -5, "norte": -2}, "frequência": "d", "demanda": 7, "janela_de_tempo": {"início": 8, "fim": 16}},
    {"id": 7, "posição": {"leste": -4, "norte": -7}, "frequência": "d", "demanda": 3, "janela_de_tempo": {"início": 9, "fim": 16}},
    {"id": 8, "posição": {"leste": 6, "norte": 0}, "frequência": "d", "demanda": 4, "janela_de_tempo": {"início": 9, "fim": 18}},
    {"id": 9, "posição": {"leste": 3, "norte": -6}, "frequência": "d", "demanda": 6, "janela_de_tempo": {"início": 6, "fim": 12}},
    {"id": 10, "posição": {"leste": -1, "norte": -3}, "frequência": "d", "demanda": 5, "janela_de_tempo": {"início": 11, "fim": 19}},
    {"id": 11, "posição": {"leste": 0, "norte": -6}, "frequência": "alt", "demanda": 4, "janela_de_tempo": {"início": 4, "fim": 11}},
    {"id": 12, "posição": {"leste": 6, "norte": 4}, "frequência": "alt", "demanda": 7, "janela_de_tempo": {"início": 2, "fim": 12}},
    {"id": 13, "posição": {"leste": 2, "norte": 5}, "frequência": "alt", "demanda": 3, "janela_de_tempo": {"início": 13, "fim": 20}},
    {"id": 14, "posição": {"leste": -2, "norte": 8}, "frequência": "alt", "demanda": 4, "janela_de_tempo": {"início": 5, "fim": 10}},
    {"id": 15, "posição": {"leste": 6, "norte": 10}, "frequência": "alt", "demanda": 5, "janela_de_tempo": {"início": 6, "fim": 13}},
    {"id": 16, "posição": {"leste": 1, "norte": 8}, "frequência": "alt", "demanda": 6, "janela_de_tempo": {"início": 10, "fim": 15}},
    {"id": 17, "posição": {"leste": -3, "norte": 1}, "frequência": "alt", "demanda": 8, "janela_de_tempo": {"início": 4, "fim": 11}},
    {"id": 18, "posição": {"leste": -6, "norte": 5}, "frequência": "alt", "demanda": 5, "janela_de_tempo": {"início": 7, "fim": 11}},
    {"id": 19, "posição": {"leste": 2, "norte": 9}, "frequência": "alt", "demanda": 7, "janela_de_tempo": {"início": 5, "fim": 10}},
    {"id": 20, "posição": {"leste": -6, "norte": -5}, "frequência": "alt", "demanda": 6, "janela_de_tempo": {"início": 2, "fim": 11}},
    {"id": 21, "posição": {"leste": 5, "norte": -4}, "frequência": "alt", "demanda": 6, "janela_de_tempo": {"início": 13, "fim": 20}}
]

distancias = [[calcula_distancia_produtores(i, j) for j in produtores] for i in produtores]

velocidade_coleta = 1 # mil  litros por minuto
velocidade_deslocamento = 80 # km por hora

## Variáveis de Decisão

In [6]:
Model = ConcreteModel()

# Criando a lista de IDs dos produtores
produtores_ids = [p["id"] for p in produtores]

# Definindo o modelo
Model = ConcreteModel()

# Definindo as variáveis
Model.x = Var(produtores_ids, produtores_ids, within=Boolean)

# Variáveis que representam o número de produtores visitados antes de visitar o produtor i nos dias 1 e 2
Model.y = Var(produtores_ids, within=NonNegativeIntegers)


# Definindo as variáveis para o tempo de início da coleta
Model.t = Var(produtores_ids, within=NonNegativeReals)

## Definindo o objetivo 
### (Minimizar a distância percorrida nos dois dias)


In [7]:
# Cálculo da distância total percorrida no dia 1
distancia = sum(sum(Model.x[p1['id'], p2['id']] * calcula_distancia_produtores(p1, p2) 
                     for p2 in produtores) for p1 in produtores)

# Função objetivo do modelo, define o objetivo de minimizar distância total
Model.obj = Objective(expr=(distancia), sense=minimize)

## Definindo as restrições


In [8]:
Model.constraints = ConstraintList()
M  = 24000

n = len(produtores)

# Garante 2 visitas aos produtores maiores e 1 visita para alternados entre o par de dias de coleta
for p in produtores:
    i = p['id']
    Model.constraints.add(Model.x[i, i] == 0)
    Model.constraints.add(sum(Model.x[i, j] for j in produtores_ids if j != i) == 1)
    Model.constraints.add(sum(Model.x[j, i] for j in produtores_ids if j != i) == 1)

    inicio_janela = p['janela_de_tempo']['início']
    Model.constraints.add(Model.t[i] >= inicio_janela)
    for q in produtores:
        j = q['id']
        if i != j:
            tempo_total_ij = calcula_tempo_coleta(p)  + distancias[i-1][j-1]/ velocidade_deslocamento
            Model.constraints.add(Model.t[j] >= (Model.t[i]  + tempo_total_ij) - M * (1 - Model.x[i, j]))
        if j != 1:
            Model.constraints.add(Model.y[j] >= Model.y[i] + Model.x[i, j] * (n - 1) + Model.x[j, i] * (n - 3) - (n - 2))

# Garante que a rota comece no ponto de partida (zero)
Model.constraints.add(Model.y[1] == 0) 
for i in produtores_ids[1:]:
    Model.constraints.add(Model.y[i] >= 0)


## Resolvendo:

In [ ]:
solver = SolverFactory('glpk', solver_io='lp')
#solver.options['tmlim'] = 900

results = solver.solve(Model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\CAU~1\AppData\Local\Temp\tmpv6js0ogf.glpk.raw --wglp C:\Users\CAU~1\AppData\Local\Temp\tmpm5yg0i58.glpk.glp
 --cpxlp C:\Users\CAU~1\AppData\Local\Temp\tmpja5wg51v.pyomo.lp
Reading problem data from 'C:\Users\CAU~1\AppData\Local\Temp\tmpja5wg51v.pyomo.lp'...
C:\Users\CAU~1\AppData\Local\Temp\tmpja5wg51v.pyomo.lp:7553: warning: lower bound of variable 'x423' redefined
C:\Users\CAU~1\AppData\Local\Temp\tmpja5wg51v.pyomo.lp:7553: warning: upper bound of variable 'x423' redefined
945 rows, 483 columns, 3783 non-zeros
462 integer variables, 441 of which are binary
7994 lines were read
Writing problem data to 'C:\Users\CAU~1\AppData\Local\Temp\tmpm5yg0i58.glpk.glp'...
6599 lines were written
GLPK Integer Optimizer, v4.65
945 rows, 483 columns, 3783 non-zeros
462 integer variables, 441 of which are binary
Preprocessing...
20 constraint coefficient(s) were reduced
862 rows, 461 columns, 3680 non-zero

Time used: 420.0 secs.  Memory used: 48.8 Mb.
+237174: mip =     not found yet >=   7.173566716e+02        (94468; 277)
+238410: mip =     not found yet >=   7.173566716e+02        (95086; 277)
+239614: mip =     not found yet >=   7.173566716e+02        (95688; 277)
+240826: mip =     not found yet >=   7.173566716e+02        (96294; 277)
+242038: mip =     not found yet >=   7.173566716e+02        (96900; 277)
+243262: mip =     not found yet >=   7.173566716e+02        (97512; 277)
+244462: mip =     not found yet >=   7.173566716e+02        (98112; 277)
+245668: mip =     not found yet >=   7.173566716e+02        (98715; 277)
+246866: mip =     not found yet >=   7.173566716e+02        (99314; 277)
+248058: mip =     not found yet >=   7.173566716e+02        (99910; 277)
+249204: mip =     not found yet >=   7.173566716e+02        (100483; 277)
+250308: mip =     not found yet >=   7.173566716e+02        (101035; 277)
Time used: 480.0 secs.  Memory used: 51.6 Mb.
+251462: mip =    

+338032: mip =     not found yet >=   7.173566716e+02        (144897; 277)
+338876: mip =     not found yet >=   7.173566716e+02        (145319; 277)
+339716: mip =     not found yet >=   7.173566716e+02        (145739; 277)
+340544: mip =     not found yet >=   7.173566716e+02        (146153; 277)
+341384: mip =     not found yet >=   7.173566716e+02        (146573; 277)
+342214: mip =     not found yet >=   7.173566716e+02        (146988; 277)
+343042: mip =     not found yet >=   7.173566716e+02        (147402; 277)
+343874: mip =     not found yet >=   7.173566716e+02        (147818; 277)
Time used: 1052.4 secs.  Memory used: 69.8 Mb.
+344694: mip =     not found yet >=   7.173566716e+02        (148228; 277)
+345510: mip =     not found yet >=   7.173566716e+02        (148636; 277)
+346326: mip =     not found yet >=   7.173566716e+02        (149044; 277)
+347144: mip =     not found yet >=   7.173566716e+02        (149453; 277)
+347958: mip =     not found yet >=   7.173566716e+02

+412914: mip =     not found yet >=   7.173566716e+02        (182338; 277)
+413566: mip =     not found yet >=   7.173566716e+02        (182664; 277)
+414188: mip =     not found yet >=   7.173566716e+02        (182975; 277)
+414808: mip =     not found yet >=   7.173566716e+02        (183285; 277)
Time used: 4370.9 secs.  Memory used: 88.5 Mb.
+415410: mip =     not found yet >=   7.173566716e+02        (183586; 277)
+416010: mip =     not found yet >=   7.173566716e+02        (183886; 277)
+416608: mip =     not found yet >=   7.173566716e+02        (184185; 277)
+417216: mip =     not found yet >=   7.173566716e+02        (184489; 277)
+417832: mip =     not found yet >=   7.173566716e+02        (184797; 277)
+418458: mip =     not found yet >=   7.173566716e+02        (185110; 277)
+419068: mip =     not found yet >=   7.173566716e+02        (185415; 277)
+419628: mip =     not found yet >=   7.173566716e+02        (185695; 277)
+420238: mip =     not found yet >=   7.173566716e+02

+472894: mip =     not found yet >=   7.173566716e+02        (212328; 277)
+473436: mip =     not found yet >=   7.173566716e+02        (212599; 277)
+473988: mip =     not found yet >=   7.173566716e+02        (212875; 277)
+474546: mip =     not found yet >=   7.173566716e+02        (213154; 277)
+475090: mip =     not found yet >=   7.173566716e+02        (213426; 277)
+475610: mip =     not found yet >=   7.173566716e+02        (213686; 277)
+476140: mip =     not found yet >=   7.173566716e+02        (213951; 277)
+476662: mip =     not found yet >=   7.173566716e+02        (214212; 277)
Time used: 4911.0 secs.  Memory used: 100.5 Mb.
+477178: mip =     not found yet >=   7.173566716e+02        (214470; 277)
+477682: mip =     not found yet >=   7.173566716e+02        (214722; 277)
+478210: mip =     not found yet >=   7.173566716e+02        (214986; 277)
+478738: mip =     not found yet >=   7.173566716e+02        (215250; 277)
+479296: mip =     not found yet >=   7.173566716e+0

+525144: mip =     not found yet >=   7.173566716e+02        (238453; 277)
+525598: mip =     not found yet >=   7.173566716e+02        (238680; 277)
+525982: mip =     not found yet >=   7.173566716e+02        (238872; 277)
+526428: mip =     not found yet >=   7.173566716e+02        (239095; 277)
+526820: mip =     not found yet >=   7.173566716e+02        (239291; 277)
+527254: mip =     not found yet >=   7.173566716e+02        (239508; 277)
+527704: mip =     not found yet >=   7.173566716e+02        (239733; 277)
+528120: mip =     not found yet >=   7.173566716e+02        (239941; 277)
+528502: mip =     not found yet >=   7.173566716e+02        (240132; 277)
+528908: mip =     not found yet >=   7.173566716e+02        (240335; 277)
+529374: mip =     not found yet >=   7.173566716e+02        (240568; 277)
+529868: mip =     not found yet >=   7.173566716e+02        (240815; 277)
Time used: 5451.1 secs.  Memory used: 110.8 Mb.
+530350: mip =     not found yet >=   7.173566716e+0

## Resultados:

In [ ]:
print_rota_otima(Model)

distancia_total = value(Model.obj)
print("\nDistância total percorrida: {:.2f} Km".format(distancia_total))


for i in produtores_ids:
    print(f'Produtor: {i} Entrada: {value(Model.t[i])}')